# mission_to_mars
----

Written in the Python 3.7.9 Environment

By Nicole Lund 

This Jupyter Notebook scrapes Mars space data from various locations for storage in a Pymongo DB and display on a webpage.

In [1]:
# Import Dependencies
import pandas as pd 
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.19/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\nlund\.wdm\drivers\chromedriver\win32\91.0.4472.19]


# NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text.

In [3]:
# Access NASA news site
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)

In [4]:
# Collect the latest news headline and paragraph
nasa_html = browser.html
nasa_soup = BeautifulSoup(nasa_html, 'html.parser')

latest_container = nasa_soup.find('div', class_='image_and_description_container')

news_headline = latest_container.find('div', class_='content_title').find('a').text

news_teaser = latest_container.find('div', class_='article_teaser_body').text

In [5]:
print(news_headline)
print('-------------------')
print(news_teaser)

NASA's InSight Mars Lander Gets a Power Boost
-------------------
The spacecraft successfully cleared some dust off its solar panels, helping to raise its energy and delay when it will need to switch off its science instruments.


# JPL Mars Space Images - Featured Image

* Scrape the [JPL Featured Space Image Website](https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html) for the complete url string of the full size .jpg Featured Mars Image.

In [6]:
# Access JPL image site
jpl_base_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
jpl_url = jpl_base_url + 'index.html'
browser.visit(jpl_url)

In [7]:
# Collect the full url path for the full size featured image
jpl_html = browser.html
jpl_soup = BeautifulSoup(jpl_html, 'html.parser')

featured = jpl_soup.find('div', class_='floating_text_area')
jpl_relative_url = featured.find('a')['href']
featured_image_url = jpl_base_url + jpl_relative_url

In [8]:
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg


# Mars Facts

* Use Pandas to scrape the table containing facts about the planet from [Mars Facts webpage](https://space-facts.com/mars/) then convert the data to a HTML table string.

In [9]:
# Collect Mars Facts Table
facts_url = 'https://space-facts.com/mars/'
facts_df = pd.read_html(facts_url)[0]

# Clean up table
facts_df = facts_df.rename(columns={0:'Description',1:'Value'})
facts_df = facts_df.set_index('Description')
facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [10]:
# Convert facts table to html string
facts_html = facts_df.to_html()
facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

# Mars Hemispheres

* Obtain full resolution images and image names for each of Mar's hemispheres from [USGS Astrogeology site](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to a dictionary then store both dictionaries in a list.

In [37]:
# Access Astrogeology site
astropedia_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(astropedia_url)
astropedia_html = browser.html
astropedia_soup = BeautifulSoup(astropedia_html, 'html.parser')

In [40]:
# Initialize image list
hemisphere_titles = []

# Find the hemisphere titles containers
hemisphere_containers = astropedia_soup.find_all('div', class_='description')

# Collect hemisphere titles
for image_num in range(0,10):
    try:
        hemisphere_found = hemisphere_containers[image_num].h3.text
        hemisphere_titles.append(hemisphere_found)
        print(f'Found Hemisphere: {hemisphere_found}')
    except:
        print('All Hemispheres Found')

# print('----------------')
# print(hemisphere_titles)

Found Hemisphere: Cerberus Hemisphere Enhanced
Found Hemisphere: Schiaparelli Hemisphere Enhanced
Found Hemisphere: Syrtis Major Hemisphere Enhanced
Found Hemisphere: Valles Marineris Hemisphere Enhanced
All Hemispheres Found
All Hemispheres Found
All Hemispheres Found
All Hemispheres Found
All Hemispheres Found
All Hemispheres Found


In [41]:
# Navigate to each hemisphere link and collect image link
hemisphere_links = []

for hemisphere in hemisphere_titles:
    # Navigate to each hemisphere link
    browser.links.find_by_partial_text(hemisphere).click()

    # Collect URL for full size image
    hemisphere_html = browser.html
    hemisphere_soup = BeautifulSoup(hemisphere_html, 'html.parser')

    downloads = hemisphere_soup.find('div', class_='downloads')
    links = downloads.find_all('a')
    for link_num in range(0,5):
        try:
            link_title = links[link_num].text
            if link_title == 'Original':
                image_link = links[link_num]['href']
                print(f'Image Link Found: {image_link}')
                hemisphere_links.append(image_link)
        except:
            print('All Links Found')

    # Return to main page
    browser.visit(astropedia_url)

# print(hemisphere_links)

Image Link Found: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif
All Links Found
All Links Found
All Links Found
Image Link Found: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif
All Links Found
All Links Found
All Links Found
Image Link Found: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif
All Links Found
All Links Found
All Links Found
Image Link Found: https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif
All Links Found
All Links Found
All Links Found
